In [1]:
from selenium import webdriver
from time import sleep
import re
import pandas as pd

In [4]:
driver = webdriver.Chrome()

In [5]:
base_url = 'https://jumia.com.eg/smartphones/'
driver.get(base_url)

In [6]:
def close_pop(driver):
    pop_results = driver.find_elements_by_css_selector('#pop .cw .cls')
    if len(pop_results) > 0:
        pop_results[0].click()

In [7]:
last_page = driver.find_element_by_css_selector('.pg-w a:last-of-type').get_attribute('href')
last_page_num = int(re.sub("[^0-9]", "", last_page))

In [9]:
def product_info(driver):
    products = driver.find_elements_by_css_selector('article[class="prd _fb col c-prd"] a.core div.info')
    df = pd.DataFrame(columns = ['title','brand','price','old_price','discount','discount_percent','rate'])
    for product in products:
        title = ''
        if len(product.find_elements_by_css_selector('h3.name')) > 0:
            title = product.find_elements_by_css_selector('h3.name')[0].text

        brand = title.split()[0] 

        price = 0.0
        if len(product.find_elements_by_css_selector('div.prc')) > 0:
            price = product.find_elements_by_css_selector('div.prc')[0].text
            price = float(price.split()[1].strip().replace(',',''))


        old_price =  0.0
        if len(product.find_elements_by_css_selector('div.s-prc-w div.old')) > 0:
            old_price = product.find_elements_by_css_selector('div.s-prc-w div.old')[0].text
            old_price = float(old_price.split()[1].strip().replace(',',''))

        if old_price == 0 :
            discount = 0
        else:
            discount = old_price - price

        discount_percent = 0
        if len(product.find_elements_by_css_selector('div._dsct')) > 0:
            discount_percent = product.find_elements_by_css_selector('div._dsct')[0].text
            discount_percent = int(discount_percent.replace('%',''))

        rate = 0.0
        if len(product.find_elements_by_css_selector('div.rev div.stars')) > 0:
            rate = product.find_elements_by_css_selector('div.rev div.stars')[0].text
            rate = float(rate.split()[0])

        product_info = {
            'title':title ,
            'brand': brand,
            'price': price,
            'old_price':old_price ,
            'discount':discount ,
            'discount_percent':discount_percent ,
            'rate':rate
        }
        df = df.append(product_info,ignore_index = True)
    return df

In [30]:
data = pd.DataFrame(columns = ['title','brand','price','old_price','discount','discount_percent','rate'])
for page in range(1,last_page_num +1):
    driver.get(f"{base_url}/?page={str(page)}#catalog-listing")
    close_pop(driver)
    df = product_info(driver)
    data = data.append(df,ignore_index = True)

In [32]:
data.head()

,title,brand,price,old_price,discount,discount_percent,rate
0,Nokia 8.3 5G - 6.67-inch 128GB/8GB Mobile Phon...,Nokia,7999.0,10499.0,2500,24,5.0
1,Samsung Galaxy A21s - 6.5-inch 64GB/4GB Dual S...,Samsung,2799.0,3299.0,500,15,4.6
2,XIAOMI Redmi 9C - 6.53-inch 64GB/3GB Dual SIM ...,XIAOMI,2040.0,2499.0,459,18,4.9
3,XIAOMI Redmi Note 8 - 6.3-inch 64GB/4GB Mobile...,XIAOMI,2620.0,2905.0,285,10,4.5
4,Nokia 3.2 - 6.26-inch 16GB/2GB Mobile Phone - ...,Nokia,1555.0,2200.0,645,29,0.0


In [33]:
data.tail()

,title,brand,price,old_price,discount,discount_percent,rate
940,Honor 7C - 5.99-inch 32GB/3GB Dual SIM Mobile ...,Honor,2379.0,3230.0,851,26,0.0
941,Honor 8X - 6.5-inch 64GB Dual SIM 4G Mobile Ph...,Honor,3419.0,4636.0,1217,26,4.0
942,Tecno Camon CX - 5.5-inch 16GB/2GB Dual SIM 4G...,Tecno,3078.0,0.0,0,0,4.5
943,Alcatel 1 (5033D) - 5.0-inch 8GB 4G Mobile Pho...,Alcatel,699.0,0.0,0,0,3.9
944,Huawei Y9 Prime 2019 - 6.59-inch 128GB Mobile ...,Huawei,3399.0,0.0,0,0,4.5


In [35]:
data.to_csv('mobile_data.csv')

In [6]:
driver.quit()